**Problématique :**

*Quel type de délits (vol, abus de biens sociaux, discrimination, harcèlement moral, attouchements sexuels, homicide involontaire) est sujet aux plus d’appel devant les Cours d’appel, puis devant la Cour de cassation ?*

Nous considérons les arrêts rendus entre 1er janvier 2019 et le 1er janvier 2023.

**Délits pris en compte :**
- Le vol (article 331-1 du code pénal)
- Les agressions sexuelles (article 222-22 du code pénal)
- La corruption (article 431-1 du code pénal)
- Le harcèlement moral (article L. 1152-1 du code du travail)
- Les abus de biens sociaux (article L. 231-3 du code de commerce)
- L’homicide involontaire (article 221-6 du code pénal)
- La discrimination (article 225-1 du code pénal)
- Les coups et blessures (article 222-7 du code pénal)

**Plan d’attaque :**

**I. Les délits faisant l’objet de plus d’appel et de renvoi**

1. Scrapper les données de Légifrance relatives à tous les arrêts de Cours d’appel et de la Cour de cassation rendus entre le 1er janvier 2019 et le 1er janvier 2023, afin que Python puisse analyser le contenu de ces arrêts.
2. Déterminer dans ces arrêts ceux relatifs aux délits mentionnés ci-dessus, en fonction des articles y faisant référence. Cela nous permet de déterminer quels sont les délits qui font l’objet de plus d’appel, puis ceux faisant l’objet de plus de renvoi devant la Cour de cassation.
3. À partir de ces données, établir un graphe en moustache pour représenter proportionnellement (en pourcentage, et non seulement en nombre), les délits faisant l’objet du plus d’appel devant les juridictions d’appel / de renvoi devant la Cour de cassation. Ainsi, nous savons quels délits font l’objet de plus de renvoi en appel, puis en cassation.

**II. L’analyse en fonction de la readability**
1. Il convient ensuite de se concentrer sur la readability (la lisibilité) des textes, c’est-à-dire qu’il faut considérer si les articles sont plus ou moins clairs les uns par rapport aux autres. Notre hypothèse de départ est que les articles les moins clairs (avec la plus faible readibility) font l’objet de plus de renvois, puisqu'ils laissent plus de marge d’interprétation aux parties, aux avocats, et aux juges.
3. A partir du document CSV, analyser la readability de chaque article (article 331-1 du code pénal, article 222-22 du code pénal, article 431-1 du code pénal, article L. 1152-1 du code du travail, article L. 231-3 du code de commerce, article 221-6 du code pénal, article 225-1 du code pénal, article 222-7 du code pénal.
3. Présenter les résultats sous la forme d’un graphe représentant chaque article et sa readability.

In [ ]:
import pandas as pd
import re
import numpy as np
from requests_oauthlib import OAuth2Session
import requests
import json
import matplotlib.pyplot as plt

# OAuth2 authentication parameters
API_HOST = "https://sandbox-api.aife.economie.gouv.fr"
TOKEN_URL = 'https://sandbox-oauth.aife.economie.gouv.fr/api/oauth/token'
client_id = "b800ee81-4b08-4b7c-9cb6-67f51535888e"
client_secret = "0adff97c-722d-40aa-bbd6-3e32a08c6ae3"

# Obtain the token
tokencall = requests.post(TOKEN_URL, data={"grant_type": "client_credentials","client_id": client_id, "client_secret": client_secret, "scope": "openid"})
token = tokencall.json()
client = OAuth2Session(client_id, token=token)

# List of legal articles to search
articles_de_loi = [
    '311-1 du code pénal',    # Theft
    '222-22 du code pénal',   # Sexual Assault
    '433-1 du code pénal',    # Corruption
    'L. 1152-1 du code du travail', # Moral Harassment
    'L. 231-3 du code de commerce', # Abuse of Corporate Assets
    '221-6 du code pénal',    # Involuntary Homicide
    '225-1 du code pénal',    # Discrimination
    '222-7 du code pénal'     # Assault and Battery
]

# Base URL for the search
base_urls = {
    'ca': "https://sandbox-api.piste.gouv.fr/cassation/judilibre/v1.0/search?query=pénal&date_start=2019-01-01&date_end=2023-01-01&judilibre_juridiction=ca&page_size=50&page=",
    'cc': "https://sandbox-api.piste.gouv.fr/cassation/judilibre/v1.0/search?query=pénal&date_start=2019-01-01&date_end=2023-01-01&judilibre_juridiction=cc&page_size=50&page="
}

# DataFrame to store the results
df_resultats = pd.DataFrame(columns=["decision_id", "jurisdiction", "texte_decision", "article_loi"])

# Function to add a row to the DataFrame
def ajouter_ligne_df(df, decision_id, jurisdiction, texte_decision, article):
    new_row = pd.DataFrame({"decision_id": [decision_id], "jurisdiction": [jurisdiction], "texte_decision": [texte_decision], "article_loi": [article]})
    return pd.concat([df, new_row], ignore_index=True)

# Browse the result pages for the Court of Appeal and the Court of Cassation
for key, base_url in base_urls.items():
    page = 1
    while True:
        resp = client.get(base_url + str(page))
        if resp.status_code == 200:
            try:
                results = resp.json()["results"]
                for result in results:
                    decision_id = result["id"]
                    jurisdiction = result.get("jurisdiction", "").lower()
                    decision_resp = client.get(f"https://sandbox-api.piste.gouv.fr/cassation/judilibre/v1.0/decision?id={decision_id}")
                    if decision_resp.status_code == 200:
                        try:
                            decision_resp_json = decision_resp.json()
                            texte_decision = decision_resp_json.get("text", "")

                            # Check for presence of legal articles
                            for article in articles_de_loi:
                                if re.search(article.replace(' ', '\s*'), texte_decision, re.S|re.I):
                                    df_resultats = ajouter_ligne_df(df_resultats, decision_id, jurisdiction, texte_decision, article)
                        except json.JSONDecodeError:
                            print(f"JSON decoding error for decision {decision_id}")
            except json.JSONDecodeError:
                print(f"JSON decoding error for page {page}")
            page += 1
        elif resp.status_code == 416:
            # End of page range, exit loop
            break
        else:
            print(f"Request failed with status code: {resp.status_code}")
            break

df_resultats_ca = df_resultats[df_resultats['jurisdiction'] == 'ca']
df_resultats_cc = df_resultats[df_resultats['jurisdiction'] == 'cc']

# Create a DataFrame for storing the summary
df_resume = pd.DataFrame(columns=["Article_Loi", "Nombre_Decisions_ca", "Nombre_Decisions_cc","Percentage of Cassation Appeals"])

# Count and add data in df_resume
for article in articles_de_loi:
    nombre_decisions_ca = len(df_resultats_ca[df_resultats_ca["article_loi"] == article])
    nombre_decisions_cc = len(df_resultats_cc[df_resultats_cc["article_loi"] == article])
    df_resume = df_resume.append({"Article_Loi": article, "Nombre_Decisions_ca": nombre_decisions_ca, "Nombre_Decisions_cc": nombre_decisions_cc}, ignore_index=True)

# Initialize the 'Percentage of Cassation Appeals' column with NaN
df_resume['Percentage of Cassation Appeals'] = np.nan

# Calculate the ratio avoiding division by zero
for index, row in df_resume.iterrows():
    if row['Nombre_Decisions_cc'] != 0:
        df_resume.at[index, 'Percentage of Cassation Appeals'] = row['Nombre_Decisions_ca'] / row['Nombre_Decisions_cc'] * 100
    else:
        df_resume.at[index, 'Percentage of Cassation Appeals'] = 0

# Calculation of the total decisions of the Court of Cassation
total_decisions_cc = df_resume['Nombre_Decisions_cc'].sum()

# Adding a column for the percentage of Court of Cassation decisions
if total_decisions_cc != 0:
    df_resume['Percentage_Decisions_cc'] = (df_resume['Nombre_Decisions_cc'] / total_decisions_cc) * 100
else:
    df_resume['Percentage_Decisions_cc'] = 0

# Load data
df = pd.read_csv("testpenal.csv", encoding="utf8", usecols=[0], names=["Texte"])
df.fillna("", inplace=True)
df['Texte'] = df['Texte'].astype(str).str.replace(r'\d+', '', regex=True)

# Function to calculate the LIX index
def calculate_lix(text):
    sentences = text.count('.') + text.count('!') + text.count('?')
    words = len(re.findall(r'\b\w+\b', text))
    long_words = len(re.findall(r'\b\w{7,}\b', text))

    if sentences == 0 or words == 0:
        return 0
    lix = words / sentences + (long_words * 100) / words
    return lix

# Apply the function on each text
df['readability_score'] = df['Texte'].apply(calculate_lix)

# Merge df_resume with readability scores
df_resume['readability_score'] = df['readability_score']

print(df_resume)
df_resultats.to_csv("df_resultats.csv", index=False, encoding='utf-8-sig')
df_resume.to_csv("df_resume.csv", index=False, encoding='utf-8-sig')

# Plot the curves
plt.figure(figsize=(12, 6))
plt.plot(df_resume['Article_Loi'], df_resume['Percentage of Cassation Appeals'], label='Percentage of Cassation Appeals', marker='o')
plt.plot(df_resume['Article_Loi'], df_resume['readability_score'], label='Readability Score', marker='x')
plt.plot(df_resume['Article_Loi'], df_resume['Percentage_Decisions_cc'], label='Percentage of Court of Cassation Decisions', marker='^')
plt.xlabel('Crime/Article')
plt.ylabel('Values')
plt.xticks(rotation=45)
plt.title('Percentage of Cassation Appeals, Readability Score and Percentage of Court of Cassation Decisions by Crime/Article')
plt.legend()
plt.tight_layout()
plt.show()
